In [1]:
import os
import ROOT
from math import pow, sqrt

Welcome to JupyROOT 6.28/04


#### Calculate normalization factor for fake rate measurement
#### Based on M(ll) distribution

In [2]:
WORKDIR = "/home/choij/workspace/ChargedHiggsAnalysis"
ERA     = "2016preVFP"
CHANNEL = "MeasFakeEl"
HLTPATH = "Ele8"
ID      = "tight"

In [3]:
#### sample list
DataStream = ""
if "El" in CHANNEL:
    if "2016" in ERA:  DataStream = "DoubleEG"
    if "2017" in ERA:  DataStream = "SingleElectron"
    if "2018" in ERA:  DataStream = "EGamma"
if "Mu" in CHANNEL:
    DataStream = "DoubleMuon"

In [4]:
W  = ["WJets_MG"]
DY = ["DYJets", "DYJets10to50_MG"]
TT = ["TTLL_powheg", "TTLJ_powheg"]
VV = ["WW_pythia", "WZ_pythia", "ZZ_pythia"]
ST = ["SingleTop_sch_Lep", "SingleTop_tch_top_Incl", "SingleTop_tch_antitop_Incl",
      "SingleTop_tW_top_NoFullyHad", "SingleTop_tW_antitop_NoFullyHad"]
MCList = W + DY + TT + VV + ST

In [5]:
SYSTs = []
if CHANNEL == "MeasNormEl":
    SYSTs.append(("PileupReweight"))
    SYSTs.append(("L1PrefireUp", "L1PrefireDown"))
    SYSTs.append(("ElectronRecoSFUp", "ElectronRecoSFDown"))
    SYSTs.append(("HeavyTagUpUnCorr", "HeavyTagDownUnCorr"))
    SYSTs.append(("LightTagUpUnCorr", "LightTagDownUnCorr"))
    SYSTs.append(("JetResUp", "JetResDown"))
    SYSTs.append(("JetEnUp", "JetEnDown"))
    SYSTs.append(("ElectronResUp", "ElectronResDown"))
    SYSTs.append(("ElectronEnUp", "ElectronEnDown"))
    SYSTs.append(("MuonEnUp", "MuonEnDown"))
if CHANNEL == "MeasNormMu":
    SYSTs.append(("PileupReweight"))
    SYSTs.append(("L1PrefireUp", "L1PrefireDown"))
    SYSTs.append(("MuonRecoSFUp", "MuonRecoSFDown"))
    SYSTs.append(("HeavyTagUpUnCorr", "HeavyTagDownUnCorr"))
    SYSTs.append(("LightTagUpUnCorr", "LightTagDownUnCorr"))
    SYSTs.append(("JetResUp", "JetResDown"))
    SYSTs.append(("JetEnUp", "JetEnDown"))
    SYSTs.append(("ElectronResUp", "ElectronResDown"))
    SYSTs.append(("ElectronEnUp", "ElectronEnDown"))
    SYSTs.append(("MuonEnUp", "MuonEnDown"))

In [6]:
# data
file_path = f"{WORKDIR}/data/MeasFakeRateV2/{ERA}/{CHANNEL}__/DATA/MeasFakeRateV2_{DataStream}.root"
print(file_path)
assert os.path.exists(file_path)
f = ROOT.TFile.Open(file_path)
data = f.Get(f"ZEnriched_{HLTPATH}/{ID}/Central/pair/mass"); data.SetDirectory(0)
f.Close()

/home/choij/workspace/ChargedHiggsAnalysis/data/MeasFakeRateV2/2016preVFP/MeasFakeEl__/DATA/MeasFakeRateV2_DoubleEG.root


In [7]:
data.Integral()

1987.0

In [15]:
def getSumMC(syst="Central"):
    sum = 0.
    for sample in MCList:
        file_path = f"{WORKDIR}/data/MeasFakeRateV2/{ERA}/{CHANNEL}__RunSyst__/MeasFakeRateV2_{sample}.root"
        assert os.path.exists(file_path)
        f = ROOT.TFile.Open(file_path)
        try:
            h = f.Get(f"ZEnriched_{HLTPATH}/{ID}/{syst}/pair/mass");   h.SetDirectory(0)
            sum += h.Integral()
        except:
            print(sample)
            continue
    f.Close()
    print(sum)
    return sum

In [16]:
### Get central scale
sumMC = {}
sumMC["Central"] = getSumMC()
for systset in SYSTs:
    if len(systset) == 2:
        systUp, systDown = systset
        sumMC[systUp] = getSumMC(systUp)
        sumMC[systDown] = getSumMC(systDown)
    else:
        syst = systset
        sumMC[syst] = getSumMC(syst)

nan


In [10]:
# get scale values
scaleMC = {}
for syst, sum in sumMC.items():
    scaleMC[syst] = data.Integral() / sum
    
diffMC = {}
for syst, sum in scaleMC.items():
    diffMC[syst] = (scaleMC[syst] - scaleMC["Central"]) / scaleMC["Central"]


In [11]:
from pprint import pprint
pprint(diffMC)

{'Central': nan}


In [12]:
print(scaleMC["Central"])

nan
